In [1]:
import sympy as sym
from IPython.display import display
print sym.__version__

1.1.1


In [2]:
sym.init_printing(use_latex='mathjax')  # TODO: check if this is needed
x, t = sym.symbols('x t', real=True)

In [3]:
def area(dist):
    return sym.simplify(sym.integrate(dist, (x, -sym.oo, sym.oo)))

def mean(dist):
    return area(dist*x)

def EX2(dist):
    return area(dist*x**2)

def variance(dist):
    return sym.simplify(EX2(dist) - mean(dist)**2)

def mgf(dist):
    return sym.simplify(area(dist*sym.exp(x*t)))

In [4]:
def summarize(dist):
    print "Distribution:"
    display(dist)
    print "Mean:"
    display(mean(dist))
    print "Variance:"
    display(variance(dist))
    print "MGF:"
    display(mgf(dist))

summarise = summarize  # alias

In [5]:
# Define other symbols that show up
mu = sym.symbols('mu', real=True)
sigma, a, b, lamb, nu = sym.symbols('sigma a b lambda nu', positive=True)

### Normal distribution: $\mathcal{N}(x; \mu, \sigma^2)$

In [6]:
normal = (2*sym.pi*sigma**2) ** sym.Rational(-1, 2) * sym.exp(-(x-mu)**2/(2*sigma**2))

In [7]:
summarize(normal)

Distribution:


             2 
    -(-μ + x)  
    ───────────
           2   
        2⋅σ    
√2⋅ℯ           
───────────────
     2⋅√π⋅σ    

Mean:


μ

Variance:


 2
σ 

MGF:


   ⎛       2  ⎞
 t⋅⎝2⋅μ + σ ⋅t⎠
 ──────────────
       2       
ℯ              

In [8]:
sym.integrate(normal, x)

   ⎛√2⋅(-μ + x)⎞
erf⎜───────────⎟
   ⎝    2⋅σ    ⎠
────────────────
       2        

### Laplace distribution: $DoubleExp(x; \mu, b)$

In [9]:
laplace = (2*b) ** (-1) * sym.exp(-sym.Abs(x-mu)/b)

In [10]:
summarize(laplace)

Distribution:


 -│μ - x│ 
 ─────────
     b    
ℯ         
──────────
   2⋅b    

Mean:


μ

Variance:


   2
2⋅b 

MGF:


⎧                                             ⎛ ⅈ⋅π                 ⎞    
⎪          μ⋅t                                ⎜ ───                 ⎟    
⎪        -ℯ                                   ⎜  2                  ⎟    
⎪      ─────────         for periodic_argument⎝ℯ   ⋅polar_lift(t), ∞⎠ = 0
⎪       2  2                                                             
⎪      b ⋅t  - 1                                                         
⎪                                                                        
⎪∞                                                                       
⎨⌠                                                                       
⎪⎮   b⋅t⋅x - │μ - x│                                                     
⎪⎮   ───────────────                                                     
⎪⎮          b                                                            
⎪⎮  ℯ                                                                    
⎪⎮  ──────────────── dx               

In [11]:
# sym.refine(mgf(laplace), sym.Q.positive(1/b - sym.Abs(t)))  # had to abort

### Exponential distribution: $Exp(x; \lambda)$

In [12]:
expo = sym.Piecewise(
    (0, x < 0),
    (lamb * sym.exp(-lamb*x), True)
)

In [13]:
summarize(expo)

Distribution:


⎧   0     for x < 0
⎪                  
⎨   -λ⋅x           
⎪λ⋅ℯ      otherwise
⎩                  

Mean:


1
─
λ

Variance:


1 
──
 2
λ 

MGF:


⎧               λ                     │                 ⎛ ⅈ⋅π                 
⎪             ─────               for │periodic_argument⎝ℯ   ⋅polar_lift(t), ∞
⎪             λ - t                                                           
⎪                                                                             
⎪∞                                                                            
⎪⌠                                                                            
⎨⎮  ⎧      0        for x < 0                                                 
⎪⎮  ⎪                                                                         
⎪⎮  ⎨   -x⋅(λ - t)            dx                      otherwise               
⎪⎮  ⎪λ⋅ℯ            otherwise                                                 
⎪⎮  ⎩                                                                         
⎪⌡                                                                            
⎩-∞                                                 

### Gamma distribution: $Gamma(x; a, b)$

In [14]:
gamma = sym.Piecewise(
    (0, x < 0),
    (b**a / sym.gamma(a) * x**(a-1) * sym.exp(-x*b), True)
)

In [15]:
summarize(gamma)

Distribution:


⎧       0         for x < 0
⎪                          
⎪ a  a - 1  -b⋅x           
⎨b ⋅x     ⋅ℯ               
⎪───────────────  otherwise
⎪      Γ(a)                
⎩                          

Mean:


a
─
b

Variance:


a 
──
 2
b 

MGF:


⎧⎧                  -a                                                        
⎪⎪     a  -a ⎛b - t⎞              b                                           
⎪⎪    b ⋅t  ⋅⎜─────⎟         for ─── > 1                                      
⎪⎪           ⎝  t  ⎠             │t│          │                 ⎛ ⅈ⋅π         
⎪⎨                                        for │periodic_argument⎝ℯ   ⋅polar_li
⎪⎪               -a                                                           
⎪⎪ a  -a ⎛-b + t⎞    -ⅈ⋅π⋅a                                                   
⎪⎪b ⋅t  ⋅⎜──────⎟  ⋅ℯ         otherwise                                       
⎪⎩       ⎝  t   ⎠                                                             
⎪                                                                             
⎨∞                                                                            
⎪⌠                                                                            
⎪⎮  ⎧          0            for x < 0               

### Beta distribution: $Beta(x; a, b)$

In [16]:
beta = sym.Piecewise(
    (0, x < 0),
    (0, x > 1),
    (x**(a-1)*(1-x)**(b-1)/(sym.gamma(a)*sym.gamma(b)/sym.gamma(a+b)), True)
)

In [17]:
beta

⎧              0                for x > 1 ∨ x < 0
⎪                                                
⎪ a - 1         b - 1                            
⎨x     ⋅(-x + 1)     ⋅Γ(a + b)                   
⎪─────────────────────────────      otherwise    
⎪          Γ(a)⋅Γ(b)                             
⎩                                                

In [18]:
# mean(beta)  # had to abort

### Uniform distribution

In [19]:
uniform = sym.Piecewise(
    (0, x < 0),
    (0, x > 1),
    (1, True)
)

In [20]:
summarize(uniform)

Distribution:


⎧0  for x > 1 ∨ x < 0
⎨                    
⎩1      otherwise    

Mean:


1/2

Variance:


1/12

MGF:


⎧  1     for t = 0
⎪                 
⎪ t               
⎨ℯ  - 1           
⎪──────  otherwise
⎪  t              
⎩                 

### Student t distribution

In [21]:
student = (1 + ((x-mu) / sigma)**2 / nu)**(-(1+nu)/2) * sym.gamma((nu+1)/2)/(sym.gamma(nu/2)*sym.sqrt(nu*sym.pi)*sigma)

Student's t doesn't have an MGF, so we just compute the ones we want

In [22]:
student

                 ν   1         
               - ─ - ─         
                 2   2         
⎛            2⎞                
⎜    (-μ + x) ⎟         ⎛ν   1⎞
⎜1 + ─────────⎟       ⋅Γ⎜─ + ─⎟
⎜          2  ⎟         ⎝2   2⎠
⎝       ν⋅σ   ⎠                
───────────────────────────────
                   ⎛ν⎞         
          √π⋅√ν⋅σ⋅Γ⎜─⎟         
                   ⎝2⎠         

In [23]:
mean(student)

⎧                                                                ν   1    
⎪                            μ                               for ─ + ─ > 1
⎪                                                                2   2    
⎪                                                                         
⎪∞                                                                        
⎪⌠                                                                        
⎪⎮                                        ν   1                           
⎪⎮   ν                                  - ─ - ─                           
⎪⎮   ─ + 1                                2   2                           
⎨⎮   2      ν   ⎛ 2              2    2⎞         ⎛ν   1⎞                  
⎪⎮  ν     ⋅σ ⋅x⋅⎝μ  - 2⋅μ⋅x + ν⋅σ  + x ⎠       ⋅Γ⎜─ + ─⎟                  
⎪⎮                                               ⎝2   2⎠                  
⎪⎮  ──────────────────────────────────────────────────── dx    otherwise  
⎪⎮                       

In [24]:
variance(student)

                                                                              
                                                                              
  ⎛⎧                              2                                    ν   1  
  ⎜⎪                             μ                                 for ─ + ─ >
  ⎜⎪                                                                   2   2  
  ⎜⎪                                                                          
  ⎜⎪                                                            2             
  ⎜⎪⎛∞                                                         ⎞              
  ⎜⎪⎜⌠                                                         ⎟              
  ⎜⎪⎜⎮                                        ν   1            ⎟              
  ⎜⎪⎜⎮   ν                                  - ─ - ─            ⎟              
- ⎜⎨⎜⎮   ─ + 1                                2   2            ⎟              
  ⎜⎪⎜⎮   2      ν   ⎛ 2              2    2⎞        

Note that not all the simplifications have been made, the mean should be $\mu$ and variance should be $\nu \sigma^2 / (\nu - 2)$

In [25]:
sym.trigsimp(variance(student))

                                                                              
  ⎛⎧                              2                                    ν   1  
  ⎜⎪                             μ                                 for ─ + ─ >
  ⎜⎪                                                                   2   2  
  ⎜⎪                                                                          
  ⎜⎪                                                            2             
  ⎜⎪⎛∞                                                         ⎞              
  ⎜⎪⎜⌠                                                         ⎟              
  ⎜⎪⎜⎮                                        ν   1            ⎟              
  ⎜⎪⎜⎮   ν                                  - ─ - ─            ⎟              
- ⎜⎨⎜⎮   ─ + 1                                2   2            ⎟              
  ⎜⎪⎜⎮   2      ν   ⎛ 2              2    2⎞         ⎛ν   1⎞   ⎟              
  ⎜⎪⎜⎮  ν     ⋅σ ⋅x⋅⎝μ  - 2⋅μ⋅x + ν⋅σ  + x ⎠       ⋅

Why doesn't it cancel?!?!

In [26]:
-mu**2 + (mu**2*nu-2*mu**2+nu*sigma**2)/(nu-2)

        2        2      2
   2   μ ⋅ν - 2⋅μ  + ν⋅σ 
- μ  + ──────────────────
             ν - 2       

In [27]:
sym.simplify(-mu**2 + (mu**2*nu-2*mu**2+nu*sigma**2)/(nu-2))

    2
 ν⋅σ 
─────
ν - 2

In [28]:
sym.simplify(sym.cos(x/2)**2/(sym.cos(x)+1))

    2⎛x⎞  
 cos ⎜─⎟  
     ⎝2⎠  
──────────
cos(x) + 1

In [29]:
sym.trigsimp(sym.cos(x/2)**2/(sym.cos(x)+1))

    2⎛x⎞  
 cos ⎜─⎟  
     ⎝2⎠  
──────────
cos(x) + 1